In [1]:
!nvidia-smi
%env CUDA_VISIBLE_DEVICES=0
%env CUDA_VISIBLE_DEVICES

Fri May  7 13:42:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Tesla V1...  On   | 00000000:37:00.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA Tesla V1...  On   | 00000000:AF:00.0 Off |                    0 |
| N/A   

'0'

In [ ]:
######################################### ASR not-pretained main script #################################################

# Import general libraries
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from datetime import datetime

import editdistance

# Import stuff from other ASR modules
from asr.data import BaseDataset
from asr.data.preprocessors import SpectrogramPreprocessor, TextPreprocessor
from asr.modules import ASRModel
from asr.utils.training import batch_to_tensor, epochs, Logger
from asr.utils.text import greedy_ctc
from asr.utils.metrics import ErrorRateTracker, LossTracker

def get_Stats(ref_batch,hyp_batch):
    ref_WER_splits = [rf.split() for rf in ref_batch]
    hyp_WER_splits = [hp.split() for hp in hyp_batch]
    ref_WER_lens = [len(rf_spl) for rf_spl in ref_WER_splits]
    hyp_WER_lens = [len(hp_spl) for hp_spl in hyp_WER_splits]
    
    WER = [editdistance.eval(rf,hp) for rf, hp in zip(ref_WER_splits, hyp_WER_splits)]
    
    ref_CER_splits = [list(rf) for rf in ref_batch]
    hyp_CER_splits = [list(hp) for hp in hyp_batch]
    ref_CER_lens = [len(rf_spl) for rf_spl in ref_CER_splits]
    hyp_CER_lens = [len(hp_spl) for hp_spl in hyp_CER_splits]
    
    CER = [editdistance.eval(rf,hp) for rf, hp in zip(ref_CER_splits, hyp_CER_splits)]
    
    return WER, ref_WER_lens, hyp_WER_lens, CER, ref_CER_lens, hyp_CER_lens


""" Function: Train and test ASR model on input datasets
    Input:    2 txt-files with IDs to training and test files. One observation consists of a transcript
              (txt-feature) and audio file (wav-target).
    Output:   Return best WER (validation) and save ASR model (model.pt) """


""" Part 1: Load and preprocess data """
train_IDs = 'test-clean-cut'
test_IDs = 'dev-clean'


train_source = train_IDs
val_source = test_IDs

# BLACK BOX
spec_preprocessor = SpectrogramPreprocessor(output_format='NFT', sample_rate=4000)
text_preprocessor = TextPreprocessor()
preprocessor = [spec_preprocessor, text_preprocessor]

train_dataset = BaseDataset(source=train_source, preprocessor=preprocessor, sort_by=0)
val_dataset = BaseDataset(source=val_source, preprocessor=preprocessor, sort_by=0)

# Data loader
train_loader = DataLoader(train_dataset, num_workers=4, pin_memory=True, collate_fn=train_dataset.collate, batch_size=16)
val_loader = DataLoader(val_dataset, num_workers=4, pin_memory=True, collate_fn=val_dataset.collate, batch_size=16)

""" Part 2: Setup model and loss """
# Create instance of model
asr_model = ASRModel(input_size=40).cuda()
print(asr_model)
print("Trainable parameters:", sum(p.numel() for p in asr_model.parameters() if p.requires_grad))

# Define loss, optimizer and learning rate scheduler
ctc_loss = nn.CTCLoss(reduction='none').cuda()#reduction='sum'
optimizer = torch.optim.Adam(asr_model.parameters(), lr=3e-4)
lr_scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=5e-5)

""" Part 3: Train and evaluate model """
# Variables to create and store performance metrics
wer_metric = ErrorRateTracker(word_based=True)
cer_metric = ErrorRateTracker(word_based=False)
ctc_metric = LossTracker()

train_logger = Logger('Training', ctc_metric, wer_metric, cer_metric)
val_logger = Logger('Validation', ctc_metric, wer_metric, cer_metric)

def forward_pass(batch, training=False):
    (x, x_sl), (y, y_sl) = batch_to_tensor(batch)  # For CPU: change 'cuda' to 'cpu'
    
    logits, output_sl = asr_model.forward(x, x_sl.cpu())
    log_probs = F.log_softmax(logits, dim=2)
    loss = ctc_loss(log_probs, y, output_sl, y_sl)
    
    hyp_encoded_batch = greedy_ctc(logits, output_sl)
    hyp_batch = text_preprocessor.decode_batch(hyp_encoded_batch)
    ref_batch = text_preprocessor.decode_batch(y, y_sl)
    
    wer_metric.update(ref_batch, hyp_batch)
    cer_metric.update(ref_batch, hyp_batch)
    ctc_metric.update(loss.sum().item(), weight=output_sl.sum().item())

    if not training:
        WER, ref_WER_lens, hyp_WER_lens, CER, ref_CER_lens, hyp_CER_lens = get_Stats(ref_batch,hyp_batch)
        CTC_loss = [ctc.item() for ctc in loss]
        return CTC_loss, WER, ref_WER_lens, hyp_WER_lens, CER, ref_CER_lens, hyp_CER_lens
    else:
        return loss

# Run 200 epochs
model_name = f"{train_IDs}vs{test_IDs}{datetime.now().strftime('Y%Y-m%m-d%d-H%H-M%M')}" 
with open(f"./results/{model_name}.csv", "a+") as o_f:
    for epoch in epochs(200):
        # Set PyTorch in training mode
        asr_model.train()

        # Train model on training set
        print("Running training:")
        for batch, files in train_logger(train_loader):
            loss = forward_pass(batch, training=True)
            optimizer.zero_grad()
            loss.sum().backward()
            optimizer.step()

        # Set PyTorch in test mode
        asr_model.eval()

        # Test model using test set
        print("Running evaluation:")
        N = 1
        wer_sum = 0
        with torch.no_grad():
            for batch, files in val_logger(val_loader):
                CTC_loss, WER, ref_WER_lens, hyp_WER_lens, CER, ref_CER_lens, hyp_CER_lens = forward_pass(batch, training=False)
                wer_sum += np.sum(WER)
                N += len(WER)
                for i, flac_file in enumerate(files):
                    idx = os.path.splitext(os.path.basename(flac_file))[0]
                    print(f"{epoch}\t{idx}\t{CTC_loss[i]}\t{WER[i]}\t{ref_WER_lens[i]}\t{hyp_WER_lens[i]}\t{CER[i]}\t{ref_CER_lens[i]}\t{hyp_CER_lens[i]}", file=o_f)

        best_wer = np.inf
        if wer_sum/N < best_wer:
            best_wer = wer_sum/N
            torch.save(asr_model.state_dict(), f"./results/best_{model_name}.pt")

        if epoch >= 100:
            lr_scheduler.step()

ASRModel(
  (conv1d_layer): Conv1d(40, 256, kernel_size=(5,), stride=(2,), padding=(2,))
  (lstm_block): LSTM(256, 256, num_layers=2, dropout=0.4, bidirectional=True)
  (output_layer): Linear(in_features=512, out_features=29, bias=True)
)
Trainable parameters: 2695965

Epoch 1
Running training:
Training [63/63, 0.5 min(s)]: Loss=1.070, WER=99.99, CER=100.14
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.857, WER=100.00, CER=100.00

Epoch 2
Running training:
Training [63/63, 0.5 min(s)]: Loss=0.842, WER=100.00, CER=100.00
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.853, WER=100.00, CER=100.00

Epoch 3
Running training:
Training [63/63, 0.5 min(s)]: Loss=0.838, WER=100.00, CER=100.00
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.848, WER=100.00, CER=100.00

Epoch 4
Running training:
Training [63/63, 0.5 min(s)]: Loss=0.833, WER=100.00, CER=100.00
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.841, WER=100.00, CER=100.00

Epoch 5

Training [63/63, 0.5 min(s)]: Loss=0.223, WER=69.69, CER=24.38
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.597, WER=95.29, CER=50.02

Epoch 44
Running training:
Training [63/63, 0.6 min(s)]: Loss=0.217, WER=69.41, CER=23.90
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.606, WER=96.91, CER=50.46

Epoch 45
Running training:
Training [63/63, 0.7 min(s)]: Loss=0.212, WER=68.64, CER=23.37
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.607, WER=97.60, CER=50.51

Epoch 46
Running training:
Training [63/63, 0.6 min(s)]: Loss=0.208, WER=68.12, CER=23.10
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.626, WER=99.46, CER=51.47

Epoch 47
Running training:
Training [63/63, 0.5 min(s)]: Loss=0.202, WER=66.77, CER=22.38
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.627, WER=99.94, CER=51.01 

Epoch 48
Running training:
Training [63/63, 0.7 min(s)]: Loss=0.194, WER=65.25, CER=21.54
Running evaluation:
Validation [169/169, 1.2 m

Training [63/63, 0.6 min(s)]: Loss=0.056, WER=27.82, CER=6.11
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.882, WER=95.23, CER=51.04

Epoch 88
Running training:
Training [63/63, 0.6 min(s)]: Loss=0.054, WER=27.19, CER=5.91
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.884, WER=94.76, CER=51.30

Epoch 89
Running training:
Training [63/63, 0.6 min(s)]: Loss=0.052, WER=26.18, CER=5.75
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.885, WER=95.09, CER=51.41

Epoch 90
Running training:
Training [63/63, 0.6 min(s)]: Loss=0.050, WER=25.14, CER=5.47
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.893, WER=95.24, CER=51.46

Epoch 91
Running training:
Training [63/63, 0.6 min(s)]: Loss=0.048, WER=24.20, CER=5.19
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=0.897, WER=94.65, CER=51.22

Epoch 92
Running training:
Training [63/63, 0.6 min(s)]: Loss=0.047, WER=23.47, CER=5.02
Running evaluation:
Validation [169/169, 1.2 min(s)]:

Training [63/63, 0.5 min(s)]: Loss=0.015, WER=7.40, CER=1.42
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=1.098, WER=95.92, CER=50.94

Epoch 132
Running training:
Training [63/63, 0.6 min(s)]: Loss=0.015, WER=7.28, CER=1.39
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=1.105, WER=95.95, CER=51.03

Epoch 133
Running training:
Training [63/63, 0.6 min(s)]: Loss=0.015, WER=7.14, CER=1.36
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=1.109, WER=94.96, CER=50.72

Epoch 134
Running training:
Training [63/63, 0.5 min(s)]: Loss=0.014, WER=6.53, CER=1.23
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=1.111, WER=95.62, CER=50.72

Epoch 135
Running training:
Training [63/63, 0.5 min(s)]: Loss=0.014, WER=6.68, CER=1.26
Running evaluation:
Validation [169/169, 1.2 min(s)]: Loss=1.111, WER=95.52, CER=50.80

Epoch 136
Running training:
Training [63/63, 0.5 min(s)]: Loss=0.014, WER=6.35, CER=1.22
Running evaluation:
Validation [169/169, 1.2 min(s)]: 